In [1]:
import pandas as pd

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Lambda

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from tqdm import tqdm

# all the functions from helpers.py
from helpers_scenario2 import *

import pandas as pd

from joblib import Parallel, delayed, parallel_backend
import multiprocessing
from multiprocessing import Value
from tqdm.auto import tqdm

# all the functions from helpers.py
from helpers_scenario2 import *

from keras.layers import Dropout, LSTM
from keras.regularizers import l1, l2
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import RandomSearch

from keras.layers import MaxPooling1D

import json
from keras.layers import Dropout, LSTM
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import RandomSearch

from keras.layers import MaxPooling1D


C:\Users\xochipilli\AppData\Local\Temp\ipykernel_13080\1464098886.py:35: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [26]:
folds = [0,1,2,3,4]

scenario = 2

for fold in folds:

    scenario_folder = f"../data/preprocessed/cleaned_and_prepro_improved/scenario_{scenario}/"


    phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)
    annotations_folder = f"../data/raw/scenario_2/fold_{fold}/test/annotations/"
    physiology_folder = f"../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_{fold}/test/physiology/" 
    zipped_files = zip_csv_files(annotations_folder, physiology_folder)

    subjects, videos = get_subs_vids(physiology_folder)
    print(subjects)
    print(videos)
    

    num_cpu_cores = multiprocessing.cpu_count()

    def process_files(annotation_file, physiology_file,):
        df_annotations = pd.read_csv(annotation_file)
        df_physiology = pd.read_csv(physiology_file)
        
        print(physiology_file)
        X, y, numeric_column_indices = preprocess(df_physiology, df_annotations, predictions_cols=['arousal','valence'], aggregate=None, window=[-10000, 10000], partition_window = 1)
        
        save_files(X, y, annotation_file, phys_folder, ann_folder)
        
        return None

    ## Process the files using the context manager
    with parallel_backend('loky', n_jobs=num_cpu_cores // 3):
        with tqdm_joblib(tqdm(total=len(zipped_files), desc="Files", leave=False)) as progress_bar:
            results = Parallel()(
                (delayed(process_files)(ann_file, phys_file) for ann_file, phys_file in zipped_files)
            )



[3, 12, 14, 19, 23, 44]
[0, 2, 9, 10, 11, 13, 14, 20]


Files:   0%|          | 0/48 [00:00<?, ?it/s]

In [22]:
splits


{'train': [31, 24, 7, 45], 'test': [32, 20]}

In [20]:
def create_cnn_lstm_model(input_shape, lstm_units=64, dropout_rate=0.4, kernel_regularizer_l1=0.0032):
    input_signal = Input(shape=input_shape)

    x = Conv1D(16, 5, padding='same')(input_signal)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = MaxPooling1D(pool_size=2)(x)

    x = LSTM(lstm_units)(x)

    x = Dense(32, activation='relu', kernel_regularizer=l1(kernel_regularizer_l1))(x)

    return Model(inputs=input_signal, outputs=x)

In [25]:
for fold in folds:

    scenario_folder = f"../data/preprocessed/cleaned_and_prepro_improved/scenario_{scenario}/"


    phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)
    annotations_folder = f"../data/raw/scenario_2/fold_{fold}/test/annotations/"
    physiology_folder = f"../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_{fold}/train/physiology/" 

    subjects, videos = get_subs_vids(physiology_folder)
    print(subjects)
    print(videos)
    
    splits = split_subjects_train_test_final_model(subjects)
    print(splits)

[0, 1, 2, 5, 7, 9, 11, 13, 15, 17, 18, 20, 24, 26, 27, 28, 29, 30, 31, 32, 34, 41, 43, 45]
[0, 2, 9, 10, 11, 13, 14, 20]
{'train': [43, 11, 0, 13, 32, 2, 34, 15, 29, 9, 5, 24, 45, 31, 41, 26], 'test': [28, 18, 30, 17, 7, 27, 1, 20]}
[0, 2, 3, 5, 7, 9, 12, 13, 14, 15, 18, 19, 20, 23, 24, 27, 29, 30, 31, 32, 34, 43, 44, 45]
[0, 2, 9, 10, 11, 13, 14, 20]
{'train': [27, 32, 14, 30, 19, 20, 18, 9, 29, 23, 0, 2, 13, 43, 5, 24], 'test': [34, 15, 7, 12, 44, 31, 3, 45]}
[0, 1, 2, 3, 7, 11, 12, 13, 14, 15, 17, 18, 19, 20, 23, 24, 26, 28, 31, 32, 41, 43, 44, 45]
[0, 2, 9, 10, 11, 13, 14, 20]
{'train': [2, 17, 11, 14, 43, 12, 7, 41, 18, 28, 24, 15, 19, 3, 1, 20], 'test': [26, 0, 23, 31, 44, 13, 45, 32]}
[1, 3, 5, 7, 9, 11, 12, 14, 17, 19, 20, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 41, 44, 45]
[0, 2, 9, 10, 11, 13, 14, 20]
{'train': [24, 41, 34, 32, 26, 23, 9, 14, 19, 31, 30, 17, 11, 7, 3, 5], 'test': [1, 29, 44, 20, 28, 12, 27, 45]}
[0, 1, 2, 3, 5, 9, 11, 12, 13, 14, 15, 17, 18, 19, 23, 26, 27, 2

In [ ]:
import os
import json

# Initialize an empty dictionary to store the fold results
fold_results = {}

# def save_test_data(y_pred, output_folder, y_test_file, test=False, y_test=None):
#     y_test_file = y_test_file.replace('npy', 'csv')
    
#     if test:
#         df = pd.read_csv(y_test_file)
#         df['valence'] = y_pred[:, 0]
#         df['arousal'] = y_pred[:, 1]  
#     else:
#         df = pd.DataFrame(y_test, columns=['valence', 'arousal'])
#         df['valence_pred'] = y_pred[:, 1]
#         df['arousal_pred'] = y_pred[:, 0] 
#         df['time'] = pd.read_csv(y_test_file).tail(df.shape[0])['time'].values         
    
#     df.to_csv(os.path.join(output_folder, os.path.basename(y_test_file)), index=False)
#     return None

scenario = 2

# Process each fold
folds = [0,1,2,3,4]

for fold in folds:
    
    output_folder = f"data/predictions/scenario_2/fold_{fold}"
    scenario_folder = f"../data/preprocessed/cleaned_and_prepro_improved/scenario_{scenario}/"
    phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)
    
    annotations_folder = f"../data/raw/scenario_2/fold_{fold}/test/annotations/"
    physiology_folder = f"../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_{fold}/train/physiology/" 
    zipped_files = zip_csv_files(annotations_folder, physiology_folder)

    subjects, videos = get_subs_vids(physiology_folder)
    print(subjects)
    print(videos)
    
    splits = split_subjects_train_test_final_model(subjects)
    print(splits)


    
    

    subjects, videos = get_subs_vids(phys_folder)

    os.makedirs(output_folder, exist_ok=True)
    
    # X_train = []
    # for train_test, subs in train_test_split.items():
    #     for v in videos:
    #         for sub in subs:
    #             file_path = os.path.join(base_path, f"sub_{sub}_vid_{v}.npy")
    #             if os.path.exists(file_path):
    #                 data = np.load(file_path)
    #                     X_train.append(data)

    X_train, x_test = load_and_concatenate_files(phys_folder, splits, videos)
    y_train, y_test = load_and_concatenate_files(ann_folder, splits, videos)

    X_train = np.transpose(X_train, (1, 0, 2))
    X_test = np.transpose(X_test, (1, 0, 2))

                
    # Extract arousal and valence values from y_train and y_test
    y_arousal_train = y_train[:, 0]
    y_valence_train = y_train[:, 1]
    y_arousal_test = y_test[:, 0]
    y_valence_test = y_test[:, 1]

    input_shape = (X_train.shape[1], X_train.shape[2])
    cnn_lstm_model = create_cnn_lstm_model(input_shape)

    # Create separate heads for valence and arousal prediction, using sigmoid activation and scaling the output
    valence_output = Dense(1, activation='sigmoid')(cnn_lstm_model.output)
    valence_output = Lambda(lambda x: x * 8 + 1, name='valence_output')(valence_output)

    arousal_output = Dense(1, activation='sigmoid')(cnn_lstm_model.output)
    arousal_output = Lambda(lambda x: x * 8 + 1, name='arousal_output')(arousal_output)

    # Combine the model
    final_model = Model(inputs=cnn_lstm_model.input, outputs=[valence_output, arousal_output])

    # Compile the model
    final_model.compile(optimizer='adam',
                        loss={'valence_output': 'mse',
                            'arousal_output': 'mse'},
                        metrics=[tf.keras.metrics.RootMeanSquaredError()])

    # Set up early stopping
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = final_model.fit(X_train, {'valence_output': y_valence_train, 'arousal_output': y_arousal_train},
                                validation_split=0.2, epochs=5, batch_size=32,
                                callbacks=[early_stopping_callback])

    # Predict
    y_pred = final_model.predict(X_test)

    # Save predictions
    save_test_data(y_pred, output_folder, ann_folder)

    # Calculate RMSE
    rmse_valence = np.sqrt(np.mean((y_pred[:, 0] - y_test[:, 0]) ** 2))
    rmse_arousal = np.sqrt(np.mean((y_pred[:, 1] - y_test[:, 1]) ** 2))

    # Save the RMSE in the fold_results dictionary
    fold_results[f"fold_{fold}"] = {
        'valence_output_root_mean_squared_error': rmse_valence,
        'arousal_output_root_mean_squared_error': rmse_arousal
    }

# Save the fold_results dictionary to a JSON file
with open('results_LSTM_folds.json', 'w') as outfile:
    json.dump(fold_results, outfile, indent=4)

# Print the final results
for fold_key, fold_value in fold_results.items():
    print(f"{fold_key}: {fold_value}")
